<a href="https://colab.research.google.com/github/rename-z/Deep-Learning/blob/master/Zero%20to%20GANs/Recurrent%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Loading MNIST Train Dataset**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

from torch.utils.data.dataloader import DataLoader

In [ ]:
train_dataset = MNIST(
    root='/content', 
    train=True, 
    transform=transforms.ToTensor(), 
    download=True
)

train_dataset

Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Dataset MNIST
    Number of datapoints: 60000
    Root location: /content
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_dataset = MNIST(
    root='/content',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: /content
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
batch_size=128

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size)

In [ ]:
class RNNModel(nn.Module) :
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim) :
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x) :
        h0 = torch.zero(self.layer_dim, x.size(0), self.hidden_dim).required_grad_()
        out, hn =  self.rnn(x, h0.detach())
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

In [ ]:
model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
len(list(model.parameters()))

6

In [ ]:
list(model.parameters())[0].size()

torch.Size([100, 28])

In [ ]:
list(model.parameters())[2].size()

torch.Size([100])

In [ ]:
list(model.parameters())[1].size()

torch.Size([100, 100])

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
seq_dim = 28

iter = 0
num_epochs = 6
for epoch in range(num_epochs) :
    for i, (images, labels) in enumerate(train_loader) :
        model.train()
        image = image.view(-1, seq_dim, input_dim).required_grad_()
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        iter += 1 

        if iter % 500 == 0 :
            